# HOW MUCH HAS AVERAGE TEMPERATURE CHANGED IN THE LAST CENTURY?



## Preface

Earth is warming up; there are a lot of studies that report temperature rise during last centuries. [According](https://archive.ipcc.ch/ipccreports/tar/wg1/index.php?idp=5) to IPCC (The Intergovernmental Panel on Climate Change)  global average surface temperature has increased by 0.6 ± 0.2° over the 20th century. In this paper we will try to check out this fact using GHCN (Global Historical Climatology Network) [dataset](https://console.cloud.google.com/marketplace/details/noaa-public/ghcn-m?project=esoteric-parsec-376408). The GHCN-Monthly is a temperature dataset that contains monthly mean temperatures and is used for operational climate monitoring activities. It is comprised of climate records from over 7,000 stations.


## Exploring dataset

Import modules and setup connection to dataset via Bigquery:

In [15]:
import db_dtypes
import pandas as pd
from google.cloud import bigquery
import matplotlib.pyplot as plt
import seaborn as sns
# client initialization
client = bigquery.Client()
# constructing reference to dataset
dataset_ref = client.dataset('ghcn_m', project='bigquery-public-data')
# get dataset object
dataset = client.get_dataset(dataset_ref)

Check list of tables in dataset:

In [4]:
for table in client.list_tables(dataset):
    print(table.table_id)

ghcnm_tavg
ghcnm_tavg_stations
ghcnm_tmax
ghcnm_tmax_stations
ghcnm_tmin
ghcnm_tmin_stations


Let's take a look on ghcnm_tavg table:

In [16]:
ghcnm_tavg_ref = dataset_ref.table('ghcnm_tavg')  # get table reference
ghcnm_tavg = client.get_table(ghcnm_tavg_ref)
client.list_rows(ghcnm_tavg, max_results=5).to_dataframe()

,id,year,element,value1,dmflag1,qcflag1,dsflag1,value2,dmflag2,qcflag2,...,qcflag10,dsflag10,value11,dmflag11,qcflag11,dsflag11,value12,dmflag12,qcflag12,dsflag12
0,10160355000,1931,TAVG,-9999,None,None,None,1035,None,None,...,None,1,1615,None,None,1,1125,None,None,1
1,10160355000,1933,TAVG,-9999,None,None,None,-9999,None,None,...,None,1,1545,None,None,1,1195,None,None,1
2,10160355000,1935,TAVG,925,None,None,1,1155,None,None,...,None,None,1555,None,None,1,1315,None,None,1
3,10160355000,1941,TAVG,1215,None,None,1,-9999,None,None,...,None,None,-9999,None,None,None,-9999,None,None,None
4,10160355000,1966,TAVG,-9999,None,None,None,-9999,None,None,...,None,None,1395,None,None,0,1165,None,None,0


Table contains 12 value_n columns (presumably one for each month), but they don't look like temperature numbers. So we should refer to table schema:

In [18]:
ghcnm_tavg.schema[2:4]

[SchemaField('element', 'STRING', 'NULLABLE', None, 'element type, monthly mean temperature="TAVG" monthly maximum temperature="TMAX" monthly minimum temperature="TMIN"', (), None),
 SchemaField('value1', 'INTEGER', 'NULLABLE', None, 'monthly value (MISSING=-9999). Temperature values are in hundredths of a degree Celsius, but are expressed as whole integers (e.g. divide by 100.0 to get whole degrees Celsius).', (), None)]

As we can see, table contains monthly mean temperature with values in hundredths of a degree Celsius, so we can treat them correctly

## Annual mean temperature change in 20th century


### Average temperature in 1900 and 2000

Let's compare average year temperature in 1900 and 2000. First we'll create CTE expression that constructs table with annual mean temperature:

In [42]:
query_annual_mean = """
          WITH annual_mean AS(
          SELECT id,year,(value1+value2+value3+value4+value5+value6+
                             value7+value8+value9+value10+value11+value12)
                             /100.0/12 AS mean_temperature
         FROM `bigquery-public-data.ghcn_m.ghcnm_tavg`
         WHERE value1 <> -9999 AND
               value2 <> -9999 AND
               value3 <> -9999 AND
               value4 <> -9999 AND
               value5 <> -9999 AND
               value6 <> -9999 AND
               value7 <> -9999 AND
               value8 <> -9999 AND
               value9 <> -9999 AND
               value10 <> -9999 AND
               value11 <> -9999 AND
               value12 <> -9999)
          """
query_view = """{}
                SELECT * 
                FROM annual_mean
                LIMIT 3
             """.format(query_annual_mean)
query_job = client.query(query_view)
query_job.to_dataframe().head()


,id,year,mean_temperature
0,10160355000,1967,17.741667
1,10160355000,1968,18.166667
2,10160355000,1969,17.550000


Now we can calculate mean temperature in 1900 and 2000:

In [41]:
query_1 = """{}
             SELECT year, AVG(mean_temperature) AS mean_temperature
             FROM annual_mean
             WHERE year IN (1900,2000)
             GROUP BY year
           """.format(query_annual_mean)
query_job = client.query(query_1)
query_job.to_dataframe().head()

,year,mean_temperature
0,1900,11.660771
1,2000,13.042306


### Moving average

We have got about 1.4° of temperature rise. This is much bigger than 0.6 ± 0.2° reported by IPCC,so we should assume that 1900 or 2000 was not an 'average' year. This problem could be addressed by using moving average over decade:

In [9]:
query_q1_2 = """
              WITH moving_avg AS(
              SELECT year, AVG(AVG(value1/100.0)) OVER 
                  (ORDER BY year DESC 
                  ROWS BETWEEN 5 PRECEDING AND 5 FOLLOWING) AS mean_temperature
                  
              FROM `bigquery-public-data.ghcn_m.ghcnm_tavg`
              WHERE value1 <> -9999
              GROUP BY year)
              SELECT year, mean_temperature, number_of_stations
              FROM moving_avg
              WHERE year IN (1900,2000)
              """
query_job = client.query(query_q1_2)
query_job.to_dataframe().head()

,year,mean_temperature,number_of_stations
0,2000,4.774124,2716
1,1900,0.835871,1408


In [ ]:
query = """
        SELECT year,AVG(value1/100.0 ) AS mean_temperature,COUNT(1) as number_of_stations
        FROM `bigquery-public-data.ghcn_m.ghcnm_tavg`
        WHERE  value1 <> -9999 AND
               year BETWEEN 1900 AND 2000
        GROUP BY year
           """
query_job = client.query(query)
data = query_job.to_dataframe()
sns.scatterplot(data=data, x="year", y="mean_temperature",
                size="number_of_stations", legend=False, sizes=(20, 200))
plt.show()

In [4]:
query_q1_3 = """
              WITH moving_avg AS(
              SELECT year, AVG(AVG(value1/100.0)) OVER 
                  (ORDER BY year DESC ROWS BETWEEN 10 PRECEDING AND 10 FOLLOWING) as mean_temperature,
                  COUNT(1) as number_of_stations
              FROM `bigquery-public-data.ghcn_m.ghcnm_tavg`
              WHERE value1 <> -9999 AND
                    id IN (SELECT id 
                     FROM `bigquery-public-data.ghcn_m.ghcnm_tavg`
                     WHERE year = 1900 AND
                           value1 <> -9999 AND
                           id IN 
                       (SELECT id
                        FROM `bigquery-public-data.ghcn_m.ghcnm_tavg`
                        WHERE year = 2000 AND
                              value1 <> -9999))
              GROUP BY year)
              SELECT year, mean_temperature, number_of_stations
              FROM moving_avg
              WHERE year IN (1900,2000)
              """
query_job = client.query(query_q1_3)
query_job.to_dataframe().head()

,year,mean_temperature,number_of_stations
0,2000,0.742227,838
1,1900,-0.103731,838
